In [2]:
%matplotlib inline

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
import json

In [3]:
fname = 'sp-pos-quot-dep-2021-03-28-19h20-nat.csv'
d = pd.read_csv(fname, 
                sep = ';', 
                parse_dates = ['jour'], 
                dtype = {'P': int, 'T': int},
                skiprows = [1])
d.sample(3)

,jour,cl_age90,P,T,pop
617,2020-07-08,9,41,5728,7.763206e+06
286,2020-06-08,0,559,45144,6.711500e+07
2707,2021-01-14,9,665,8142,7.763206e+06


In [5]:
d['incidence'] = d['P'] / d['pop'] * 100000
d['positivite'] = d['P'] / d['T'] * 100
d.sample(3)

,jour,cl_age90,P,T,pop,incidence,positivite
1083,2020-08-19,49,605,19561,8.592886e+06,7.040708,3.092889
3012,2021-02-10,89,1006,16352,3.245281e+06,30.998860,6.152153
1344,2020-09-12,19,640,14271,8.336486e+06,7.677095,4.484619


In [6]:
ages = d['cl_age90'].unique()

In [7]:
tic1 = time.time()
d['incidence hebdo'] = np.zeros(len(d))
for age in ages:
    d2 = d[(d['cl_age90'] == age)]
    d['temp'] = d2.apply(lambda x : (d2[ (d2['jour'] <= x['jour']) 
                                                    & (d2['jour'] > x['jour'] - np.timedelta64(1,'W'))]
                                                ['incidence']
                                                .sum()), axis = 1)
    d['incidence hebdo'] = d['incidence hebdo'] + d['temp'].fillna(0)
toc1 = time.time()
print('{:.2f} s'.format(-tic1+toc1))

2.03 s


In [9]:
d = d.drop(columns = ['temp'])
d.sample(3)

,jour,cl_age90,P,T,pop,incidence,positivite,incidence hebdo
2215,2020-11-30,39,2238,33306,8.295372e+06,26.978899,6.719510,124.057126
925,2020-08-05,9,82,4142,7.763206e+06,1.056265,1.979720,5.860981
3290,2021-03-08,9,1599,22629,7.763206e+06,20.597161,7.066154,93.865863


In [10]:
fname2 = fname[:-4] + '-processed.csv'
d.to_csv(fname2, sep = ';', index = False)